In [ ]:
import pandas as pd
import numpy as np
import time
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from tqdm import tqdm

base_path = './'

Mounted at /content/drive


In [ ]:
load_path = os.path.join(base_path, 'data/processed/energy_df.csv')
energy_df = pd.read_csv(load_path, encoding='utf-8-sig')

load_path = os.path.join(base_path, 'data/processed/region_station_df.csv')
region_station_df = pd.read_csv(load_path, encoding='utf-8-sig')

load_path = os.path.join(base_path, 'data/processed/weather_df.csv')
weather_df = pd.read_csv(load_path, encoding='utf-8-sig')

In [ ]:
def remove_extreme_spikes(df, threshold_ratio=15):
    """
    Remove buildings with extreme energy usage spikes using Max/Median ratio.
    Target: Buildings where Max usage is > 15x of Median usage.
    """
    print(f">> [Step1] Data Cleaning (Threshold: Max > {threshold_ratio} * Median)...")

    original_pnu = df['PNU'].unique()
    original_count = len(original_pnu)

    building_stats = df.groupby('PNU')['energy_usage'].agg(['max', 'median'])

    building_stats['median'] = building_stats['median'].replace(0, np.nan)
    building_stats['ratio'] = building_stats['max'] / building_stats['median']

    normal_buildings = building_stats[building_stats['ratio'] <= threshold_ratio].index

    df_clean = df[df['PNU'].isin(normal_buildings)].copy()

    removed_count = original_count - len(normal_buildings)
    removal_rate = (removed_count / original_count) * 100

    print(">> Data Cleaning Complete.")
    print(f"   - Original Buildings : {original_count:,}")
    print(f"   - Normal Buildings   : {len(normal_buildings):,}")
    print(f"   - Removed Outliers   : {removed_count:,} ({removal_rate:.2f}%)")
    print("-" * 50)

    return df_clean

clean_energy_df = remove_extreme_spikes(energy_df, threshold_ratio=15)

Data Cleaning (Threshold: Max > 15 * Median)...
>> Data Cleaning Complete.
   - Original Buildings : 1,320,041
   - Normal Buildings   : 1,312,516
   - Removed Outliers   : 7,525 (0.57%)
--------------------------------------------------


In [ ]:
def idh_lookup(weather_df, Tb_range=range(10, 30)):
    """
    Pre-calculate IDH (Heating + Cooling Degree Hours) for all stations and base temperatures.
    This Lookup Table accelerates the modeling process by avoiding redundant calculations.
    """
    print(">> [Step 2] Generating IDH Lookup Table...")
    print(f"   - Base Temperature Range: {min(Tb_range)}C ~ {max(Tb_range)}C")

    lookup_list = []

    for stn_id, group in tqdm(weather_df.groupby('STN_ID'), desc="Processing Stations"):

        group = group.sort_values('time')
        temps = group['temp'].values
        dates = group['yyyymm'].values

        for Tb in Tb_range:
            # diff = T_outdoor - T_base
            diff = temps - Tb

            # IDH logic: Heating Load + Cooling Load
            # If diff > 0 (Hotter than Tb) -> Cooling Load (CDH)
            # If diff < 0 (Colder than Tb) -> Heating Load (HDH)
            # I assume that IDH is same in both case(Cool, Heat) to simplify the model and avoid overfitting.
            idh_vals = np.abs(diff)

            # Create a temporary dataframe for grouping
            temp_df = pd.DataFrame({'yyyymm': dates, 'IDH': idh_vals})
            monthly_sum = temp_df.groupby('yyyymm')['IDH'].sum().reset_index()

            # Add metadata
            monthly_sum['STN_ID'] = stn_id
            monthly_sum['Tb'] = Tb

            lookup_list.append(monthly_sum)

    # Combine all results
    idh_table = pd.concat(lookup_list, ignore_index=True)

    print(">> Lookup Table Generated.")
    print(f"   - Total Rows: {len(idh_table):,}")
    print("-" * 50)

    return idh_table

idh_df = idh_lookup(weather_df)

>> [Step 2] Generating IDH Lookup Table...
   - Base Temperature Range: 10C ~ 29C


Processing Stations: 100%|██████████| 97/97 [00:04<00:00, 21.37it/s]


>> Lookup Table Generated.
   - Total Rows: 23,280
--------------------------------------------------


In [ ]:
idh_df

,yyyymm,IDH,STN_ID,Tb
0,202405,5887.8,90,10
1,202406,8861.2,90,10
2,202407,12499.4,90,10
3,202408,13067.8,90,10
4,202409,8675.8,90,10
...,...,...,...,...
23275,202412,18498.8,296,29
23276,202501,20097.2,296,29
23277,202502,18162.0,296,29
23278,202503,14615.8,296,29


In [ ]:
def run_idh_modeling(energy_df, idh_table, region_station_df, save_path, batch_size=10000):
    """
    Run the final IDH Modeling.
    """
    print(f">> [Step 3] Starting Optimized Modeling (Batch Size: {batch_size})...")
    print("   - Merging Station IDs...")

    # Data preparation
    station_map = region_station_df[['SIGUNGU_NM', 'STN_ID']].drop_duplicates()
    energy_merged = pd.merge(energy_df, station_map, on='SIGUNGU_NM', how='left')
    energy_merged = energy_merged.dropna(subset=['STN_ID'])

    # Resume logic
    if os.path.exists(save_path):
        try:
            processed_df = pd.read_csv(save_path, usecols=['PNU'])
            processed_pnus = set(processed_df['PNU'].unique())
            print(f"   - Resuming... (Already processed {len(processed_pnus):,} buildings)")
            energy_merged = energy_merged[~energy_merged['PNU'].isin(processed_pnus)]
        except:
            print("   - Warning: Could not read existing file. Starting from scratch.")

    unique_stations = energy_merged['STN_ID'].unique()
    print(f"   - Target Buildings: {energy_merged['PNU'].nunique():,}")

    results_buffer = []

    # Iterate by Station
    for stn_id in unique_stations:
        stn_idh_subset = idh_table[idh_table['STN_ID'] == stn_id]
        if stn_idh_subset.empty: continue

        # Make a wide table
        idh_wide = stn_idh_subset.pivot(index='yyyymm', columns='Tb', values='IDH')
        available_Tbs = idh_wide.columns.values

        # Filter buildings for this station
        stn_buildings = energy_merged[energy_merged['STN_ID'] == stn_id]

        # Iterate by Building
        for pnu, b_data in tqdm(stn_buildings.groupby('PNU'), desc=f"Stn {stn_id}"):
            if len(b_data) < 12: continue

            # Set index to yyyymm for quick alignment
            b_data = b_data.set_index('yyyymm')
            common_months = b_data.index.intersection(idh_wide.index)

            if len(common_months) < 12: continue

            # Extract Arrays (Numpy)
            y = b_data.loc[common_months, 'energy_usage'].values
            # Matrix of IDH values [Months x Tbs]
            X_matrix = idh_wide.loc[common_months].values

            best_r2 = -np.inf
            best_params = None

            # Loop over Tbs (Pure Numpy Calculation - Ultra Fast)
            for i, Tb in enumerate(available_Tbs):
                x = X_matrix[:, i] # i-th column is IDH for Tb

                # Simple Linear Regression
                # Slope(U) = Cov(x,y) / Var(x)
                # Intercept = mean(y) - Slope * mean(x)
                # Using polyfit for stability
                try:
                    slope, intercept = np.polyfit(x, y, 1)

                    y_pred = slope * x + intercept
                    ss_res = np.sum((y - y_pred) ** 2)
                    ss_tot = np.sum((y - np.mean(y)) ** 2)

                    if ss_tot == 0: r2 = 0
                    else: r2 = 1 - (ss_res / ss_tot)

                    if r2 > best_r2:
                        best_r2 = r2
                        best_params = {
                            'PNU': pnu,
                            'STN_ID': stn_id,
                            'Tb_opt': Tb,
                            'U_coeff': slope,
                            'E_base': intercept,
                            'R2': r2,
                            'Sample_N': len(common_months)
                        }
                except:
                    continue

            if best_params:
                results_buffer.append(best_params)

            # Batch Save
            if len(results_buffer) >= batch_size:
                temp_df = pd.DataFrame(results_buffer)
                if not os.path.exists(save_path):
                    temp_df.to_csv(save_path, index=False, mode='w')
                else:
                    temp_df.to_csv(save_path, index=False, mode='a', header=False)
                results_buffer = []

    # Final Save
    if results_buffer:
        temp_df = pd.DataFrame(results_buffer)
        if not os.path.exists(save_path):
            temp_df.to_csv(save_path, index=False, mode='w')
        else:
            temp_df.to_csv(save_path, index=False, mode='a', header=False)

    print(">> All Modeling Complete!")

# RUN
save_file_path = os.path.join(base_path, 'results/UBEM_Result_Final.csv')
run_idh_modeling(clean_energy_df, idh_df, region_station_df, save_file_path)

>> [Step 3] Starting Optimized Modeling (Batch Size: 10000)...
   - Merging Station IDs...
   - Target Buildings: 1,312,516


Stn 244: 100%|██████████| 2063/2063 [00:06<00:00, 338.17it/s]


>> All Modeling Complete!


In [ ]:
# Delete duplicates created by resuming

load_path = os.path.join(base_path, 'results/UBEM_Result_Final.csv')
final_df = pd.read_csv(load_path)

print(f"Count : {len(final_df):,}")
final_df = final_df.drop_duplicates(subset=['PNU'], keep='last')
print(f"Count after duplicates drop: {len(final_df):,}")

save_path = os.path.join(base_path, 'results/UBEM_Result_Final.csv')
final_df.to_csv(save_path, index=False, encoding='utf-8-sig')

Count : 1,687,934개
Count after duplicates drop: 1,233,505개
